In [1]:
import numpy as np
from PIL import Image
import cv2
import tensorflow as tf
import time
import requests  # For sending HTTP alerts
from datetime import datetime

In [2]:
from PytorchWildlife.models import detection as pw_detection

# Initialize with a valid version (e.g., 'MDV6-yolov10-c')
detection_model = pw_detection.MegaDetectorV6(
    pretrained=True,
    version="MDV6-yolov10-c"  # Choose from the list below
)

# Run detection
img_path2 = "C:/Users/Roshan Yadav/Downloads/roshan profile photo.jpg"
img_path = "C:/Users/Roshan Yadav/OneDrive/Desktop/wildlife images/Playful-Leopard-Cub-By-Will-Burrard-Lucas.jpeg"
detection_result = detection_model.single_image_detection(img_path)

Ultralytics 8.3.102  Python-3.9.13 torch-2.6.0+cpu CPU (11th Gen Intel Core(TM) i5-11320H 3.20GHz)
YOLOv10n summary (fused): 125 layers, 2,695,586 parameters, 0 gradients, 8.2 GFLOPs

0: 640x640 3 animals, 351.1ms
Speed: 29.4ms preprocess, 351.1ms inference, 2.5ms postprocess per image at shape (1, 3, 640, 640)


In [38]:
detection_result

{'img_id': 'C:/Users/Roshan Yadav/OneDrive/Desktop/wildlife images/Playful-Leopard-Cub-By-Will-Burrard-Lucas.jpeg',
 'detections': Detections(xyxy=array([[     940.65,      2.3651,        1480,      1072.2],
        [     490.69,      404.07,      807.65,       786.6],
        [     188.46,      672.25,      254.56,      766.67]], dtype=float32), mask=None, confidence=array([    0.96793,     0.92481,     0.23016], dtype=float32), class_id=array([0, 0, 0]), tracker_id=None, data={}),
 'labels': ['animal 0.97', 'animal 0.92', 'animal 0.23']}

In [3]:

# Load SpeciesNet Keras model
model_path = "C:/Users/Roshan Yadav/Downloads/speciesnet-keras-v4.0.0b-v3/full_image_88545560_22x8_v12_epoch_00153.keras"
speciesnet_model = tf.keras.models.load_model(model_path, compile=False)

# Load labels of SpeciesNet Keras model
labels_path = "C:/Users/Roshan Yadav/Downloads/speciesnet-keras-v4.0.0b-v3/full_image_88545560_22x8_v12_epoch_00153.labels.txt"
with open(labels_path, 'r') as f:
    class_labels = [line.strip() for line in f.readlines()]

From C:\Users\Roshan Yadav\speciesnet-env\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.

From C:\Users\Roshan Yadav\speciesnet-env\lib\site-packages\keras\src\backend.py:1398: The name tf.executing_eagerly_outside_functions is deprecated. Please use tf.compat.v1.executing_eagerly_outside_functions instead.



From C:\Users\Roshan Yadav\speciesnet-env\lib\site-packages\keras\src\layers\normalization\batch_normalization.py:979: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.

From C:\Users\Roshan Yadav\speciesnet-env\lib\site-packages\keras\src\layers\normalization\batch_normalization.py:979: The name tf.nn.fused_batch_norm is deprecated. Please use tf.compat.v1.nn.fused_batch_norm instead.



In [62]:
for i in class_labels:
    if "blank" in i:
        print(i)


f1856211-cfb7-4a5b-9158-c0f72fd09ee6;;;;;;blank


In [4]:
# Alert configuration
ALERT_COOLDOWN = 60  # 1 minute cooldown between alerts for same species
last_alert_time = {}

def send_alert(species,location):
    current_time = time.time()
    
    # Check if we need to send alert 
    if (species not in last_alert_time or current_time - last_alert_time[species] > ALERT_COOLDOWN):
    
        url = "https://field-project-i1h7.onrender.com//send-alert" # Replace with your Node.js endpoint
        timestamp = datetime.now().strftime("%d/%m/%Y %H:%M:%S")
        data = {
            "species": species,
            "location" : location,
            "time" : timestamp
        }
        try:
            response = requests.post(url, json=data)
            last_alert_time[species] = current_time
            print(f"[ALERT SENT] {species.capitalize()} — Status: {response.status_code}")
        except Exception as e:
            print(f"[ERROR] Failed to send alert: {e}")

In [5]:
def preprocess_for_speciesnet(cropped_img):
    """Convert OpenCV crop to SpeciesNet's expected format"""
    img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (480, 480))
    img = img / 255.0
    return np.expand_dims(img, axis=0)

def classify_with_speciesnet(cropped_img):
    """Run SpeciesNet inference on a cropped image"""
    processed_img = preprocess_for_speciesnet(cropped_img)
    preds = speciesnet_model.predict(processed_img, verbose=0)
    class_idx = np.argmax(preds[0])
    species = class_labels[class_idx].split(";")[-1].lower()
    return species, float(preds[0][class_idx])

In [29]:

# Load image
img_path = "C:/Users/Roshan Yadav/OneDrive/Desktop/wildlife images/photo-1575550959106-5a7defe28b56.jpg"
img = cv2.imread(img_path)

# Detection
detections = detection_model.single_image_detection(img)

# Process detections and draw boxes
for box, conf in zip(detections['detections'].xyxy, detections['detections'].confidence):
    if conf > 0.5:  # Confidence threshold
        x_min, y_min, x_max, y_max = map(int, box)
        
        # Classification
        cropped_img = img[y_min:y_max, x_min:x_max]
        species, species_conf = classify_with_speciesnet(cropped_img)
        print(f"Detected {species} (confidence: {species_conf:.2f})")
        
        # Draw bounding box
        cv2.rectangle(img, (x_min, y_min), (x_max, y_max),(0, 255, 0), 2)
        
        # Draw label with confidences
        label = f"{species} {conf:.2f}/{species_conf:.2f}"
        cv2.putText(img, label, (x_min, y_min-5), 
                   cv2.FONT_HERSHEY_SIMPLEX, 0.6,(0, 255, 0), 2)

# Display results
cv2.imshow("Wildlife Detection", img)
cv2.waitKey(0)
cv2.destroyAllWindows()

# To save the output:
# cv2.imwrite("output.jpg", img)


0: 640x640 (no detections), 138.1ms
Speed: 16.8ms preprocess, 138.1ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)


'06/04/2025 16:39:23'

In [10]:

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_skip = 4
    frame_id = 0
    skip = {"jaguar","leopard"}
    
    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        success, frame = cap.read()
        if not success:
            break

        # Process frame
        small_frame = cv2.resize(frame, (640, 360))
        detections = detection_model.single_image_detection(small_frame)

        # Track detected species in current frame
        current_species = set()

        for box, conf, cls_id in zip(detections['detections'].xyxy, detections['detections'].confidence, detections['detections'].class_id):
            if cls_id == 0 and conf > 0.1:  # Only high-confidence detections
                print("Class ID : ",cls_id)
                x_min, y_min, x_max, y_max = map(int, box)
                cropped_img = small_frame[y_min:y_max, x_min:x_max]
                
                if cropped_img.size == 0:
                    continue

                # Classification
                species, species_conf = classify_with_speciesnet(cropped_img)
                # if species in skip:
                #     continue
                current_species.add(species)  # Track unique species

                # Visualization
                cv2.rectangle(small_frame, (x_min, y_min), (x_max, y_max), (0,255,0), 2)
                cv2.putText(small_frame, f"{species[:12]} {conf:.1f}", 
                            (x_min, y_min-5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0,255,0), 2)
                


        # Send alerts for detected species
        for species in current_species:
            if species in skip:
                send_alert(species, "Mumbai")

        # Display
        cv2.imshow("Wildlife Detection", small_frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        frame_id += frame_skip + 1

    cap.release()
    cv2.destroyAllWindows()

# Example usage
path = "C:/Users/Roshan Yadav/Downloads/videoplayback (1).mp4"
process_video(path)  # 0 for webcam, or path to video file


0: 640x640 1 person, 712.2ms
Speed: 94.5ms preprocess, 712.2ms inference, 7.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 238.3ms
Speed: 8.1ms preprocess, 238.3ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 208.7ms
Speed: 7.0ms preprocess, 208.7ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 214.6ms
Speed: 10.3ms preprocess, 214.6ms inference, 0.5ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 227.1ms
Speed: 9.5ms preprocess, 227.1ms inference, 0.6ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 1 person, 213.3ms
Speed: 6.8ms preprocess, 213.3ms inference, 0.7ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 184.6ms
Speed: 10.6ms preprocess, 184.6ms inference, 0.4ms postprocess per image at shape (1, 3, 640, 640)

0: 640x640 (no detections), 186.1ms
Speed: 6.6ms preprocess, 186.1ms inference, 0.5ms postpro

In [ ]:


def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_skip = 4  # Skip more frames for speed
    frame_id = 0

    while cap.isOpened():
        cap.set(cv2.CAP_PROP_POS_FRAMES, frame_id)
        success, frame = cap.read()
        if not success:
            break

        # Resize frame for faster processing
        frame = cv2.resize(frame, (640, 360))

        # Detection
        detections = detection_model.single_image_detection(frame)

        # Process detections
        for box, conf in zip(detections['detections'].xyxy, detections['detections'].confidence):
            if conf > 0.5:
                x_min, y_min, x_max, y_max = map(int, box)

                cropped_img = frame[y_min:y_max, x_min:x_max]
                if cropped_img.size == 0:
                    continue

                species, species_conf = classify_with_speciesnet(cropped_img)

                # Draw bounding box
                color = (0, 255, 0)
                cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), color, 2)
                label = f"{species} {conf:.2f}/{species_conf:.2f}"
                cv2.putText(frame, label, (x_min, y_min - 5), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        # Display smaller preview
        cv2.imshow("Wildlife Detection", cv2.resize(frame, (640, 360)))

        # Exit on 'q'
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

        # Skip next N frames
        frame_id += frame_skip + 1

    cap.release()
    cv2.destroyAllWindows()

# Example usage
video_path = "C:/Users/Roshan Yadav/Downloads/videoplayback.mp4"  # Change to your video path
process_video(0)

In [ ]:
import cv2
import numpy as np
import tensorflow as tf
from concurrent.futures import ThreadPoolExecutor

executor = ThreadPoolExecutor(max_workers=2)

def preprocess_for_speciesnet(cropped_img):
    img = cv2.cvtColor(cropped_img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (480, 480))
    img = img / 255.0
    return np.expand_dims(img, axis=0)

def classify_with_speciesnet(cropped_img):
    processed_img = preprocess_for_speciesnet(cropped_img)
    preds = speciesnet_model.predict(processed_img, verbose=0)
    class_idx = np.argmax(preds[0])
    species = class_labels[class_idx].split(";")[-1].lower()
    return species, float(preds[0][class_idx])

def process_video(video_path):
    cap = cv2.VideoCapture(video_path)
    frame_skip = 4  # Process every 3rd frame

    future_results = []
    species_results = []

    while cap.isOpened():
        for _ in range(frame_skip):
            success, frame = cap.read()
            if not success:
                break
        
        if not success:
            break

        detections = detection_model.single_image_detection(frame)
        species_results.clear()

        # Submit classification tasks in parallel
        for box, conf in zip(detections['detections'].xyxy, detections['detections'].confidence):
            if conf > 0.5:
                x_min, y_min, x_max, y_max = map(int, box)
                cropped_img = frame[y_min:y_max, x_min:x_max]

                if cropped_img.size == 0:
                    continue

                future = executor.submit(classify_with_speciesnet, cropped_img)
                future_results.append((future, box, conf))

        # Get results and draw
        for future, box, conf in future_results:
            species, species_conf = future.result()
            x_min, y_min, x_max, y_max = map(int, box)
            cv2.rectangle(frame, (x_min, y_min), (x_max, y_max), (0, 255, 0), 2)
            label = f"{species} {conf:.2f}/{species_conf:.2f}"
            cv2.putText(frame, label, (x_min, y_min - 5),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 255, 0), 2)
        future_results.clear()

        cv2.imshow("Wildlife Detection", frame)
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

video_path = "C:/Users/Roshan Yadav/Downloads/videoplayback.mp4"  # Change to your video path
process_video(video_path)